# Sunburst diagram - Game Based

Notebook used to preprocess the data in order to upload it to MongoDb. THe preprocessing takes long (there are several possible patterns with length between 2 and 7, even if pruning is performed).

Cell to create the connection to neo4j

In [1]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import streamlit as st
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    

    def find_pattern(self, query_string):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_pattern, query_string)
            for row in result:
                return row["cnt"]
            
    def find_games(self, query_string):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_pattern, query_string)
            return [r["cnt"] for r in result]

    @staticmethod
    def _find_and_return_pattern(tx, query_string):
        result = tx.run(query_string+" as cnt")
        return [row for row in result]

2022-04-30 12:48:50.081 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


Function to get the name of alle teams that have played euro20

In [2]:
def all_teams():
    return ['Austria', 'Belgium', 'Croatia','Czech_Republic','Denmark','England','Finland',
     'France','Germany','Hungary','Italy','Netherlands','MACEDONIA_REPUBLIC_OF','Poland',
     'Portugal','Russia','Scotland','Slovakia','Spain','Sweden','Switzerland','Turkey','Ukraine','Wales']

Function used to produce the cypher query strings for pattern matching

In [3]:
def cypherify(string, team = None, game = None):
    letters = list(string)

    if team:
        query = "MATCH p=(A:"+team+")"
        for i in range(len(string) - 1):
            query += "-[p" + str(i) + ":PASS]->(" + letters[i + 1] + ":"+team+")"
    else:
        query = "MATCH (A)"
        for i in range(len(string) - 1):
            query += "-[p" + str(i) + ":PASS]->(" + letters[i + 1] + ")"

    query += "\nWHERE "

    #correct order
    first = True
    for i in range(len(string) - 2):
        if first:
            query += "p" + str(i) + ".order + 1 = p" + str(i + 1) + ".order"
            first = False
        else:
            query += " and p" + str(i) + ".order + 1 = p" + str(i + 1) + ".order"

    #same possession
    first = True
    for i in range(len(string) - 2):
        if first:
            query += " and p" + str(i) + ".possession = p" + str(i + 1) + ".possession"
            first = False
        else:
            query += " and p" + str(i) + ".possession = p" + str(i + 1) + ".possession"

    #same match
    first = True
    for i in range(len(string) - 2):
        if first:
            query += " and p" + str(i) + ".match_id = p" + str(i + 1) + ".match_id"
            first = False
        else:
            query += " and p" + str(i) + ".match_id = p" + str(i + 1) + ".match_id"

    #different players
    unorderedPairGenerator = ((x, y) for x in set(letters) for y in set(letters) if y > x)
    if string != "AB":
        query += " and " + " and ".join([x + ".name <>" + " " +y + ".name" for x, y in list(unorderedPairGenerator)])
    else:
        query +=  " and ".join([x + ".name <>" + " " +y + ".name" for x, y in list(unorderedPairGenerator)])
    
    if game:
        query+= " and p0.match_id = "+str(game)
    
    query += "\nRETURN COUNT(p)"
    return query

In [8]:
string = cypherify("ABA", "ITALY", 1234)
print("Example query for pattern ABA in game 1234 by Italy: \n\n"+string)

Example query for pattern ABA in game 1234 by Italy: 

MATCH p=(A:ITALY)-[p0:PASS]->(B:ITALY)-[p1:PASS]->(A:ITALY)
WHERE p0.order + 1 = p1.order and p0.possession = p1.possession and p0.match_id = p1.match_id and A.name <> B.name and p0.match_id = 1234
RETURN COUNT(p)


Function to get the number of passages given a pattern

In [9]:
def get_value(pattern, team, app, match_id):
    query = cypherify(pattern, team, match_id)
    v = app.find_pattern(query)
    #print("Pattern: "+pattern+" \t Value: "+str(v))
    result[pattern] = v
    return v

Functions to create all the possibile passing motifs and calculate the number of times they happened

In [6]:
def pass_net(i,n, vect, team, app, match_id):
    if i==1:
        vect[i-1] = "A"
        pass_net(2,n,vect, team, app, match_id)
    elif i==2:
        vect[i-1] = "B"
        v = get_value("".join(vect[:i]), team, app, match_id)
        if v>0:
            pass_net(3,n,vect, team, app, match_id)
    elif i==n+1:
        #print("".join(vect))
        return
    else:
        for l in distinct_letters(vect, i-2):
            vect[i-1] = l
            v = get_value("".join(vect[:i]), team, app, match_id)
            if v > 0:
                pass_net(i+1,n,vect, team, app, match_id)
        vect[i-1] = first_new_letter(vect,i-1)
        v = get_value("".join(vect[:i]), team, app, match_id)
        if v > 0:
            pass_net(i+1,n,vect, team, app, match_id)

def first_new_letter(vect,end):
    letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "L", "M"]
    max_letter = max(vect[:end])
    return letters[letters.index(max_letter)+1]

def distinct_letters(vect, end):
    el = set(vect[:end])
    el.discard(vect[end])
    return list(el)

Function that returns all games played by a given team

In [7]:
def get_games(team, app):
    query = 'MATCH (g:GAME) WHERE g.away_team = "' + team + '" or g.home_team = "'+team+'" RETURN g.match_id'
    v = app.find_games(query)
    return v

Visualization used in the frontend (not used in this notebook)

In [8]:
import plotly.express as px
def display_sunburst(result, team, game):
    patterns = []
    prefixes = []
    values = []
    colors = []
    for res in result[team][game]:
            patterns.append(res)
            prefixes.append(res[:-1])
            values.append(result[team][game][res])
            colors.append(res[0:4] if len(res)>=4 else  12)
    
    prefixes[0] = ""
    data = dict(
    patterns= patterns,
    prefixes= prefixes,
    values= values
    )
    
    fig = px.sunburst(data,
    names="patterns",
    parents="prefixes",
    values="values",
    color = prefixes
    )
    fig.show()

In [10]:
#neo4j
uri = ### Neo4j URI ####
user = "soccer_analytics"
password = ### Neo4j Password ####
app = App(uri, user, password)

SyntaxError: invalid syntax (3583558273.py, line 2)

In [10]:
#mongo
import pymongo
client = pymongo.MongoClient(###### MONGO LOGIN STRING ######)
db = client.soccer_analytics
col = db["sunburst_cache"]

In [22]:
#use it to skip already uploaded teams
SKIP_BEGINNING = 0

#loop through all teams and each game played by each team
for team_l in all_teams()[SKIP_BEGINNING:]:    
    #to use team names (team_n) and labels (team_l)
    if team_l == "MACEDONIA_REPUBLIC_OF":
        team_n = "North Macedonia"
    elif team_l == "Czech_Republic":
        team_n = "Czech Republic"
    else:
        team_n = team_l
    team_l = team_l.upper()
    
    #get all the games played by that team
    games = get_games(team_n, app)
    
    print(team_l+" has played the following games :"+str(games))
    
    for match_id in games:
        print("\t processing game :"+str(match_id))
        n = 6
        l = [0]*n
        result = {}
        pass_net(1, n, l, team_l, app, match_id)
        col.insert_one({
            "team":team_l,
            "match_id":match_id,
            "data":result
        })

UKRAINE has played the following games :[3794692, 3788767, 3788758, 3788746, 3795187]
	 processing game :3794692
	 processing game :3788767
	 processing game :3788758
	 processing game :3788746
	 processing game :3795187
WALES has played the following games :[3794689, 3788766, 3788755, 3788744]
	 processing game :3794689
	 processing game :3788766
	 processing game :3788755
	 processing game :3788744


2022-04-27 11:26:37.187 ERROR   neo4j: Failed to read from defunct connection IPv4Address(('00e145e7.databases.neo4j.io', 7687)) (IPv4Address(('34.126.114.186', 7687)))
2022-04-27 11:26:37.188 WARNING neo4j: Transaction failed and will be retried in 1.032966086172038s (Failed to read from defunct connection IPv4Address(('00e145e7.databases.neo4j.io', 7687)) (IPv4Address(('34.126.114.186', 7687))))
2022-04-27 11:26:38.224 ERROR   neo4j: Failed to read from defunct connection ResolvedIPv4Address(('34.126.114.186', 7687)) (IPv4Address(('34.126.114.186', 7687)))
2022-04-27 11:26:38.225 ERROR   neo4j: Unable to retrieve routing information
2022-04-27 11:26:38.226 WARNING neo4j: Transaction failed and will be retried in 1.6173206193934122s (Unable to retrieve routing information)
2022-04-27 11:26:40.933 ERROR   neo4j: Unable to retrieve routing information
2022-04-27 11:26:40.934 WARNING neo4j: Transaction failed and will be retried in 4.7677706112706435s (Unable to retrieve routing informat

Example of a invocation. This is then loaded in Mongo.

In [23]:
result

{'AB': 280,
 'ABA': 30,
 'ABAB': 4,
 'ABABA': 0,
 'ABABC': 4,
 'ABABCB': 1,
 'ABABCA': 0,
 'ABABCD': 2,
 'ABAC': 17,
 'ABACB': 0,
 'ABACA': 3,
 'ABACAB': 2,
 'ABACAC': 0,
 'ABACAD': 1,
 'ABACD': 9,
 'ABACDB': 1,
 'ABACDC': 0,
 'ABACDA': 0,
 'ABACDE': 5,
 'ABC': 185,
 'ABCB': 21,
 'ABCBC': 4,
 'ABCBCB': 0,
 'ABCBCA': 0,
 'ABCBCD': 4,
 'ABCBA': 3,
 'ABCBAB': 1,
 'ABCBAC': 0,
 'ABCBAD': 1,
 'ABCBD': 7,
 'ABCBDB': 1,
 'ABCBDC': 0,
 'ABCBDA': 1,
 'ABCBDE': 3,
 'ABCA': 4,
 'ABCAB': 0,
 'ABCAC': 0,
 'ABCAD': 3,
 'ABCADB': 0,
 'ABCADC': 0,
 'ABCADA': 0,
 'ABCADE': 2,
 'ABCD': 111,
 'ABCDB': 2,
 'ABCDBC': 0,
 'ABCDBD': 0,
 'ABCDBA': 0,
 'ABCDBE': 1,
 'ABCDC': 11,
 'ABCDCB': 1,
 'ABCDCD': 3,
 'ABCDCA': 0,
 'ABCDCE': 4,
 'ABCDA': 7,
 'ABCDAB': 0,
 'ABCDAC': 0,
 'ABCDAD': 0,
 'ABCDAE': 6,
 'ABCDE': 59,
 'ABCDEB': 4,
 'ABCDEC': 2,
 'ABCDED': 5,
 'ABCDEA': 0,
 'ABCDEF': 29}